In [181]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv
/kaggle/input/tpdatostest/test.csv


In [182]:
import pandas as pd
test = pd.read_csv("../input/tp2datostrain/test.csv")
display(test.shape)
train = pd.read_csv("../input/tp2datostrain/train.csv")
display(train.shape)


(60000, 22)

(240000, 23)

### Arreglo algunas cosas del DF
#### Swapeo metros totales y cubiertos cuando los cubiertos son mayores

In [199]:
####Data frame a usar
df = test
df["fecha"] = pd.to_datetime(df["fecha"])

In [201]:
df["swapear"] = df["metrostotales"] < df["metroscubiertos"]
df_metros_arreglados = df.copy()
aux = df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metrostotales"]
if (not aux.empty):
    df_metros_arreglados.loc[df_metros_arreglados["swapear"],"metrostotales"] = \
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"]
    df_metros_arreglados.loc[df_metros_arreglados["swapear"], "metroscubiertos"] = aux
df_metros_arreglados.drop(columns="swapear", inplace=True)
(df_metros_arreglados["metrostotales"] < df_metros_arreglados["metroscubiertos"]).value_counts()
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,swapear
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,19.408668,-99.246767,2013-07-20,0.0,0.0,0.0,0.0,0.0,False
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,113851.0,21.032480,-89.592424,2015-10-24,0.0,0.0,0.0,0.0,0.0,False
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,23620.0,19.332829,-99.152913,2015-05-30,0.0,0.0,0.0,0.0,1.0,False
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,129347.0,16.860487,-99.878383,2015-04-02,0.0,0.0,0.0,0.0,0.0,False
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,57125.0,19.640482,-99.127273,2013-08-15,0.0,0.0,0.0,1.0,1.0,True


#### Agrego precio por metro cuadrado? Esta filtrando datos del precio?

#### Agrego cuantas publicaciones hubo en ese dia


In [202]:
df["nro_publicaciones_dia"] = df.groupby(pd.Grouper(key='fecha', freq='D'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_semana"] = df.groupby(pd.Grouper(key='fecha', freq='W'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_5_dias_habiles"] = df.groupby(pd.Grouper(key='fecha', freq='B'))["id"].transform(lambda x: len(x.index))
df["nro_publicaciones_mes"] = df.groupby(pd.Grouper(key='fecha', freq='M'))["id"].transform(lambda x: len(x.index))

In [193]:
train = df
display(train.shape)
features = ['tipodepropiedad', 'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
            'provincia', 'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
           'nro_publicaciones_dia','nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles', 'nro_publicaciones_mes']
train_y = train.precio.copy()
train_x = train[features].copy()
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y,train_size=0.2, test_size=0.8, random_state = 0)

(240000, 28)

In [188]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [189]:
transformador_numeros = SimpleImputer() #usa promedio
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
cols_numericas = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
                 'gimnasio','centroscomercialescercanos','usosmultiples','piscina','escuelascercanas',
           'nro_publicaciones_dia','nro_publicaciones_semana', 'nro_publicaciones_5_dias_habiles', 'nro_publicaciones_mes']
cols_categoricas = ['tipodepropiedad', 'provincia']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, cols_numericas),
        ('cat', transformador_categorias, cols_categoricas)
    ],)#remainder = "passthrough"

In [190]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=2000, learning_rate=0.05, max_depth=10, subsample_by_tree=0.5, colsample_by_col=0.5, min_child_weight=3)
from catboost import CatBoostRegressor
model2 = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.05, loss_function='RMSE', task_type="GPU")

In [191]:
#my_pipeline = make_pipeline(preprocessor, model, verbose = True)
pipeline2 = make_pipeline(preprocessor, model2, verbose = True)

In [192]:
#my_pipeline.fit(train_x, train_y)
pipeline2.fit(train_x, train_y)

[Pipeline] . (step 1 of 2) Processing columntransformer, total=  36.3s
0:	learn: 2091661.7713082	total: 13.1ms	remaining: 13.1s
1:	learn: 2033890.4468317	total: 25.2ms	remaining: 12.6s
2:	learn: 1980140.6421176	total: 37.4ms	remaining: 12.4s
3:	learn: 1928718.5046277	total: 49.4ms	remaining: 12.3s
4:	learn: 1881473.3881015	total: 61.1ms	remaining: 12.2s
5:	learn: 1838452.7404975	total: 72.7ms	remaining: 12.1s
6:	learn: 1797099.7311817	total: 84.4ms	remaining: 12s
7:	learn: 1759014.3196566	total: 96ms	remaining: 11.9s
8:	learn: 1722843.6774901	total: 108ms	remaining: 11.9s
9:	learn: 1689873.3470801	total: 120ms	remaining: 11.8s
10:	learn: 1659043.4624636	total: 132ms	remaining: 11.8s
11:	learn: 1629903.3773359	total: 143ms	remaining: 11.8s
12:	learn: 1601526.9629344	total: 155ms	remaining: 11.8s
13:	learn: 1576494.8726288	total: 167ms	remaining: 11.7s
14:	learn: 1553532.7806949	total: 178ms	remaining: 11.7s
15:	learn: 1531928.1698600	total: 190ms	remaining: 11.7s
16:	learn: 1511070.5408

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0),
                                                  ['antiguedad', 'habitaciones',
                                                   'garages', 'banos',
                                                   'metroscubiert

In [194]:
#preds = my_pipeline.predict(test_x)
test_x= df[features].copy()

preds2 = pipeline2.predict(test_x)

In [198]:
display(preds2.shape)
display(test_x.shape)

(192000,)

(192000, 17)

In [105]:
from sklearn.metrics import mean_absolute_error
#print("MAE XGBoost: ", mean_absolute_error(test_y, preds))
print("MAE CatBoost: ", mean_absolute_error(test_y, preds2))

MAE CatBoost:  685693.4166634508


In [106]:
from sklearn.model_selection import cross_val_score
SCORE = "mean_absolute_error"
def cross_validation(pipeline, X, y, nro_folds):
    score = -1 * cross_val_score(pipeline, X, y,
                              cv=nro_folds,
                              scoring='neg_mean_absolute_error')
    print("Puntajes: ", end="")
    for p in score:
        print(p, ", ", end="")
    
    print("\nPromedio: ", np.mean(score))
    
cross_validation(pipeline2, train_x, train_y, 5)
#cross_validation(my_pipeline, train_x, train_y, 5)

[Pipeline] . (step 1 of 2) Processing columntransformer, total=   0.7s
0:	learn: 2091501.9355760	total: 125ms	remaining: 2m 5s
1:	learn: 2036374.4497466	total: 230ms	remaining: 1m 54s
2:	learn: 1982960.7611223	total: 332ms	remaining: 1m 50s
3:	learn: 1932449.2176518	total: 428ms	remaining: 1m 46s
4:	learn: 1886603.1936923	total: 524ms	remaining: 1m 44s
5:	learn: 1843836.3782022	total: 622ms	remaining: 1m 43s
6:	learn: 1803124.1467631	total: 716ms	remaining: 1m 41s
7:	learn: 1764824.8388190	total: 810ms	remaining: 1m 40s
8:	learn: 1729294.8510944	total: 909ms	remaining: 1m 40s
9:	learn: 1694919.3888104	total: 1s	remaining: 1m 39s
10:	learn: 1663258.5260170	total: 1.11s	remaining: 1m 40s
11:	learn: 1632584.3493414	total: 1.21s	remaining: 1m 39s
12:	learn: 1605538.1654214	total: 1.3s	remaining: 1m 38s
13:	learn: 1581242.1129837	total: 1.4s	remaining: 1m 38s
14:	learn: 1557658.8422877	total: 1.49s	remaining: 1m 38s
15:	learn: 1535896.6680976	total: 1.59s	remaining: 1m 37s
16:	learn: 151514

In [180]:
test.shape

(60000, 27)

In [177]:
res = pd.DataFrame(preds2, index=test.id, columns=['precio'])
res.reset_index(inplace=True)
res.columns = ["id", "target"]
display(res.head())
# RMSLE=1.0249284784393988 ?


# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(res)

ValueError: Shape of passed values is (192000, 1), indices imply (60000, 1)